<div class='alert' style='background-color: #1c1a1e; color: #f5f4f0; padding:16px 26px; border-radius:20px; font-size:40px;'><B>MONZO</b> - Bank Statement Rule Based Classification </div>
<div style='margin:0px 26px; color:#1c1a1e; font-size:16px;'>
<center>
    <img src="https://github.com/janduplessis883/Money-Mate/raw/master/images/private.png" width="250">
</center> 
    
## Introduction

In this notebook, we will explore the process of classifying transactions from a Monzo Bank statement using rule-based classification. Monzo is a popular digital bank, and its statements often contain detailed transaction data, which can be categorized for better financial management and analysis.

The primary objective of this notebook is to demonstrate how to classify bank transactions into predefined categories such as "Groceries," "Travel," "Eating Out," and others. This classification will help in understanding spending patterns, budgeting, and financial planning.

### Key Steps in this Notebook:
1. **Data Loading**: Importing the Monzo Bank statement data.
2. **Data Preprocessing**: Cleaning and preparing the data for analysis.
3. **Rule-Based Categorization**: Applying predefined rules to classify transactions into different categories.
4. **Analysis and Visualization**: Summarizing and visualizing the categorized data to gain insights into spending behavior.

By the end of this notebook, you will have a clear understanding of how rule-based classification can be applied to bank transaction data to facilitate better financial insights and management. Let's get started!
</div>

# Libraries & Data

In [45]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings
import datetime 
import os 

from money_mate.params import DATA_PATH

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
warnings.filterwarnings('ignore')

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data

In [46]:
data = pd.read_csv(f'{DATA_PATH}/monzo.csv')
data.head(2)

,Transaction ID,Date,Time,Type,Name,Emoji,Category,Amount,Currency,Local amount,Local currency,Notes and #tags,Address,Receipt,Description,Category split
0,tx_00009jGereHTyV50ElCRLl,28/05/2019,11:30:19,Faster payment,DU PLESSIS J V B,NaN,Income,150.00,GBP,150.00,GBP,BARCLAYS,NaN,NaN,BARCLAYS,NaN
1,tx_00009jGsehBRGqJ8N1IJlp,28/05/2019,14:04:51,Card payment,Boots,💊,Medical,-2.79,GBP,-2.79,GBP,💊,198-200 Fulham Palace Road,NaN,BOOTS FULHAM GBR,NaN


In [35]:
data.shape

(6442, 16)

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6442 entries, 0 to 6441
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Transaction ID   6442 non-null   object 
 1   Date             6442 non-null   object 
 2   Time             6442 non-null   object 
 3   Type             6442 non-null   object 
 4   Name             6437 non-null   object 
 5   Emoji            5394 non-null   object 
 6   Category         6442 non-null   object 
 7   Amount           6442 non-null   float64
 8   Currency         6442 non-null   object 
 9   Local amount     6442 non-null   float64
 10  Local currency   6442 non-null   object 
 11  Notes and #tags  1600 non-null   object 
 12  Address          4394 non-null   object 
 13  Receipt          7 non-null      object 
 14  Description      6238 non-null   object 
 15  Category split   0 non-null      float64
dtypes: float64(3), object(13)
memory usage: 805.4+ KB


# Exploratory Analysis

In [37]:
data['Type'].value_counts()

Card payment            5457
Faster payment           557
Pot transfer             142
Direct Debit             110
Monzo-to-Monzo            62
Monzo Paid                32
Flex                      30
Account interest          23
overdraft                 18
Bacs (Direct Credit)       5
wise_cashback              5
ledger_adjustment          1
Name: Type, dtype: int64

In [38]:
data['Category'].value_counts()

Groceries              1548
Travel                 1329
General                 515
Subscriptions           515
Eating out              509
Eating Out              401
Smoking                 292
Income                  248
Transfers               218
Other                   174
Entertainment           134
Bills                    98
Shopping                 95
Medical                  70
Transport                59
Credit Cards             38
Savings                  31
Holidays                 29
Stuff                    28
Telephone                21
Revolut                  16
PayPal                   14
Rent                     12
Online Subscription       9
Family                    7
Tax                       7
Bank Charges              5
Finances                  4
Loans                     4
Personal care             3
Gifts                     3
Holiday                   3
Charity                   2
Expenses                  1
Name: Category, dtype: int64

In [39]:
data['Type'].value_counts()

Card payment            5457
Faster payment           557
Pot transfer             142
Direct Debit             110
Monzo-to-Monzo            62
Monzo Paid                32
Flex                      30
Account interest          23
overdraft                 18
Bacs (Direct Credit)       5
wise_cashback              5
ledger_adjustment          1
Name: Type, dtype: int64

# Generating Unique Lists of Items from each Transaction **Type**

In [44]:
iterlist = ["Card payment", "Faster payment", "Monzo-to-Monzo"]

for i in iterlist:
    card = data[(data['Type'] == i)]
    card_list = list(card['Name'].unique())
    card_list.sort()
    print(i)
    print(card_list)
    print("_______________________________________________")
    

Card payment
['1104 Lhr T5 Asd U', '24/7 Zafash Pharmacy', '2theloo', 'ASOS', 'ATM', 'Abokado - Hammersmith', 'Abs Holdings', 'Abs Wholesale - Brewer', 'Ae Napier', 'Aeroporto de Lisboa', 'Airalo', 'Airbnb', 'Airtable.com/bill', 'Akfggv', 'Alamo Spur        3415', 'Alelondonrivil', 'Alma Cafe Ltd', 'Amazon', 'Amazon Fresh', 'Amazon Music', 'Amazon Prime', 'Amoret Coffee', 'Andrecampos', 'Anthropic', 'Apple', 'Apple R226 White City', 'Apple Store', 'Appliances Direct', 'Artemisa 3000', 'Audible', 'Audrey Green', 'Automate.io', 'BP', 'Bagel Bakery Bar', 'Bagel Bite', 'Bakehaus', 'Balans West', 'Balans Westfield Londo\\unit 1034\\lo', 'Barino', 'Battersea Loc', 'Battersea Park Cafe Gr', 'Bayswater Arms', 'Beigel Bake', 'Belushis Hammersmith', 'Bertotti Pure Italian', 'Big Bite', 'Big Five Duty Free', 'Black Rabbit Cafe', 'Black Sheep Coffee', 'Blanche Eatery Kensing', 'Blaze Today', 'Bloomsbury Theatre Bar', 'Booking.com', 'Bootlegger Cape Quarte', 'Boots', 'Borno Chemists St C', 'Bracken

In [41]:
data.columns

Index(['Transaction ID', 'Date', 'Time', 'Type', 'Name', 'Emoji', 'Category',
       'Amount', 'Currency', 'Local amount', 'Local currency',
       'Notes and #tags', 'Address', 'Receipt', 'Description',
       'Category split'],
      dtype='object')

## First Search Dictionary

In [ ]:
# Rules for classification based on Transaction type
transaction_type_rules = {
    'wise_cashback': 'Income',
    'Pot transfer': 'Transfer',
    'overdraft': 'Bank Charges',
    'Monzo Paid': 'Bank Charges',
    'Flex': 'Loan',
    'Account interest': 'Income',
    'Bacs (Direct Credit)': 'Income',
    # Add more transaction type rules here
}

# Rules for classification based on Transaction name
transaction_name_rules = {
    'Income': ['Salary', 'Bonus'],
    'Shopping': ['Amazon', 'eBay'],
    'Dining': ['Amazon Prime', 'McDonalds'],
    # Add more categories and names here
}

# Test Dataframe

In [19]:
# Creating a sample DataFrame for experimentation
data = {
    'Type': ['ATM', 'Deposit', 'Transfer', 'Purchase', 'Purchase', 'Transfer', 'Deposit', 'ATM', 'Purchase', 'Transfer'],
    'Name': ['', 'Salary', 'Savings', 'Amazon', 'Amazon Prime', 'Investment', 'Bonus', 'ATM Withdrawal', 'eBay', 'Retirement Fund'],
    'Amount': [100, 2000, 500, 50, 20, 1500, 500, 200, 120, 3000],
    'Date': ['2024-06-01', '2024-06-02', '2024-06-03', '2024-06-04', '2024-06-05', '2024-06-06', '2024-06-07', '2024-06-08', '2024-06-09', '2024-06-10']
}

df = pd.DataFrame(data)

df.head(10)

,Type,Name,Amount,Date
0,ATM,,100,2024-06-01
1,Deposit,Salary,2000,2024-06-02
2,Transfer,Savings,500,2024-06-03
3,Purchase,Amazon,50,2024-06-04
4,Purchase,Amazon Prime,20,2024-06-05
5,Transfer,Investment,1500,2024-06-06
6,Deposit,Bonus,500,2024-06-07
7,ATM,ATM Withdrawal,200,2024-06-08
8,Purchase,eBay,120,2024-06-09
9,Transfer,Retirement Fund,3000,2024-06-10


In [25]:
# Rules for classification based on Transaction type
transaction_type_rules = {
    'ATM': 'Cash Withdrawal',
    'Deposit': 'Income',
    'Transfer': 'Transfer',
    # Add more transaction type rules here
}

# Rules for classification based on Transaction name
transaction_name_rules = {
    'Income': ['Salary', 'Bonus'],
    'Shopping': ['Amazon', 'eBay'],
    'Dining': ['Amazon Prime', 'McDonalds'],
    # Add more categories and names here
}

In [26]:
def classify_by_type(row):
    return transaction_type_rules.get(row['Type'], 'Uncategorized')

In [27]:
def refine_by_name(row):
    if row['custom_category'] == 'Uncategorized' or row['custom_category'] in transaction_name_rules:
        for category, names in transaction_name_rules.items():
            if row['Name'] in names:
                return category
    return row['custom_category']

In [28]:
# First step: classify based on Transaction type
df['custom_category'] = df.apply(classify_by_type, axis=1)
df

,Type,Name,Amount,Date,custom_category
0,ATM,,100,2024-06-01,Cash Withdrawal
1,Deposit,Salary,2000,2024-06-02,Income
2,Transfer,Savings,500,2024-06-03,Transfer
3,Purchase,Amazon,50,2024-06-04,Uncategorized
4,Purchase,Amazon Prime,20,2024-06-05,Uncategorized
5,Transfer,Investment,1500,2024-06-06,Transfer
6,Deposit,Bonus,500,2024-06-07,Income
7,ATM,ATM Withdrawal,200,2024-06-08,Cash Withdrawal
8,Purchase,eBay,120,2024-06-09,Uncategorized
9,Transfer,Retirement Fund,3000,2024-06-10,Transfer


In [29]:
# Second step: refine classification based on Transaction name
df['custom_category'] = df.apply(refine_by_name, axis=1)
df

,Type,Name,Amount,Date,custom_category
0,ATM,,100,2024-06-01,Cash Withdrawal
1,Deposit,Salary,2000,2024-06-02,Income
2,Transfer,Savings,500,2024-06-03,Transfer
3,Purchase,Amazon,50,2024-06-04,Shopping
4,Purchase,Amazon Prime,20,2024-06-05,Dining
5,Transfer,Investment,1500,2024-06-06,Transfer
6,Deposit,Bonus,500,2024-06-07,Income
7,ATM,ATM Withdrawal,200,2024-06-08,Cash Withdrawal
8,Purchase,eBay,120,2024-06-09,Shopping
9,Transfer,Retirement Fund,3000,2024-06-10,Transfer


# Test on real data

In [47]:
# Define the classification rules
transaction_type_rules = {
    'wise_cashback': 'Income',
    'Pot transfer': 'Transfer',
    'overdraft': 'Bank Charges',
    'Monzo Paid': 'Bank Charges',
    'Flex': 'Loan',
    'Account interest': 'Income',
    'Bacs (Direct Credit)': 'Income',
}

transaction_name_rules = {
    'Shopping': [
        "Amazon",
        "Amazon Music",
        "Audible",
        "Clonezone",
        "Clonezone - Soho",
        "G-star Raw",
        "GSingh",
        "IKEA",
        "Leyland Lsdm Earl's Court",
        "Leyland SDM",
        "Robert Dyas",
        "United Shop Company",
        "eBay",
        "www.thickwall.co.uk",
        "World Duty Free",
        "Apple Store",
    ],
    'Eating Out': [
        "Taco Bell",
        "Paul Uk Hammersmith",
        "John Forrest Master Ba",
        "Five Guys",
        "Mona Lisa Cafe",
        "Wagamama",
        "Balans West",
        "Ollie’s House Limited",
        "The Monument",
        "Pret",
        "Nando’s",
        "The Grove Tavern",
        "Yoco Pickled Green",
        "Star Wraps",
        "Bagel Bakery Bar",
        "Greggs",
        "PAUL Earls Court",
        "Lions Prep",
        "Patri Takeaway",
        "Deliveroo",
        "Leon",
        "Black Rabbit Cafe",
        "GAIL's Bakery",
        "McDonald’s",
        "Starbucks",
        "Buff Meat",
        "Burger King",
        "Domino’s",
        "KFC",
        "Masala",
        "PAUL Hammersmith",
        "Pho",
        "The Bull Westfield",
        "The Grove",
        "Too Good To Go",
        "Coffee",
        "The Swan",
        "Bayswater Arms",
        "Soft Ice",
        "Old Ship, Hammersmith",
        "Riverside Studio",
    ],
    'Transport': [
        "Sendwave",
        "Zipcar",
        "Tier Mobility",
        "Lime",
        "Transport for London",
        "CMT UK Taxi Fare",
        "Uber",
        "Easi Rent",
        "easirent",
    ],
    'Subscriptions': [
        "Wordtune",
        "Automate.io",
        "Coursera",
        "Rytr - Ai Writer",
        "Netflix",
        "Zapier",
        "Notion2shee",
        "Nutt Labs + Notion Vip",
        "Google",
        "Yourdataltd",
        "Airtable.com/bill",
        "Pipedream, Inc.",
        "Superhuman",
        "Fs Revoicer",
        "Heroku",
        "Deep Learning Courses",
        "Render.com",
        "Setapp",
        "Spark",
        "Motion ",
        "Browserless",
        "www.make.com",
        "Claude",
        "WOW Presents PLUS",
        "Microsoft",
        "Loom Subscription",
        "Amazon Prime",
        "Super Publishing Co.",
        "Apple",
        "Perplexity",
        "Notion",
        "OpenAI",
        "Anthropic",
        "Motion -1 Temp Hold",
        "Brompton Super",
        "ExpressVPN",
        "Grammarly",
        "Granity-ent.com",
        "Heart Internet",
        "Jarvis - Conversion.ai",
        "Realpython",
        "Surfshark",
        "PureGym",
    ],
    'Groceries': [
        "Jms Food Store",
        "J M S Foods",
        "Sunfield Foods",
        "Amazon Fresh",
        "Lidl",
        "Waitrose & Partners",
        "M&S",
        "Tesco",
        "Sainsbury’s",
        "Co-op",
        "Deepak Self Service",
        "Chelsea Food Fayre",
        "Chelsea Food Worldsend",
        "Cumberland Food & Wine",
        "Earls Court Food And Wine",
        "J M S Food & News",
        "My Shop",
        "On The Go",
        "Chelsea Food And Wine",
    ],
    'Telephone': [
        "Airalo",
        "TELSERVE LIMITED",
        "EE",
        "plan.com",
        "giffgaff",
    ],
    'Loan': ["Credit Resource Solutions",],
    'Medical': [
        "Medicine Chest",
        "Boots",
        "NHS Prescription Prepayment",
        "Green Light Pharmacy",
        "Earls Court Chemist",
        "Superdrug",
        "Zafash Pharmacy",
    ],
    'Other': [
        "James Vokins",
        "Empriel",
        "PayPal",
        "Rm Media",
        "Leatherpr",
        "Michele Manzolillo",
        "Dipanno Dario",
        "Jan Du plessis revolut",
        "Alessandro Dei Agnoli",
        "Skrill9959",
        "James Vockins",
        "kucoin.com",
        "Patreon",
        "Cash App",
        "Wishtender.com",
        "Skrill3395",
        "ATM",
        "carlos",
        "Kostadin Milchev",
        "Revolut",
        "Koronapay Europe",
        "A Garcia",
        "Cornelioallanj",
    ],
    'Credit Cards': [
        "Capital One",
        "Vanquis Bank",
    ],
    'Gift': ['Jules Young', 'Myra Cosio',],
    'Holiday': [
        "Airbnb",
        "Booking.com",
        "Bootlegger Cape Quarte",
        "The Grey Hotel",
        "Wise Holiday",
        "Woodford Car Hire",
    ],
    'Barber': [
        "Sw5 Barbers Lt",
        "Cut And Go",
        "Old Brompton Barbers",
    ],
    'SA Investment': ['Thom',],
    'Tax': ['HMRC',],
    'Transfer': ['TWL Cattle Farming', 'Plum'],
    'Rent': ["Hampton Management", "Hampton Rent",],
    'Smoking': [
        "On The Goo",
        "The Smoking Jacket",
        "Cheyne News",
        "Deepak Self Service",
        "Lucky Me Enterprise",
        "Smoking",
    ],
    "Income": [
        "GOOD RESEARCH LTD",
        "DR BURHAN ALI ADIB AND DR ORIETTA E",
        "DU PLESSIS J V B",
        "- GOOD RESEARCH LT",
        "EARLS CT SUR",
        "Atlantic Medical",
    ],   
}

startswith_rules = {
    'Transport': ['London Taxi', 'Tier',],
    'Eating Out': ['Toogoodt',],
    'Other': ['Leather', 'Coinbase'],
}   

# Define functions for classification
def classify_by_type(row):
    return transaction_type_rules.get(row['Type'], 'Uncategorized')

def classify_by_name(row, startswith_rules):
    for category, prefixes in startswith_rules.items():
        for prefix in prefixes:
            if row['Name'].startswith(prefix):
                return category
    return None

def refine_by_name(row):
    category = classify_by_name(row, startswith_rules)
    if category:
        return category

    if row['custom_category'] == 'Uncategorized' or row['custom_category'] in transaction_name_rules:
        for category, names in transaction_name_rules.items():
            if row['Name'] in names:
                return category
    return row['custom_category']

# Load your data and apply the classification
# df['custom_category'] = df.apply(classify_by_type, axis=1)
# df['custom_category'] = df.apply(refine_by_name, axis=1)

In [ ]:
data = pd.read_csv(f'{DATA_PATH}/monzo.csv')
data.head(2)

In [48]:
data['custom_category'] = data.apply(classify_by_type, axis=1)
data['custom_category'] = data.apply(refine_by_name, axis=1)

AttributeError: 'float' object has no attribute 'startswith'

In [ ]:
data